# Questions

1) Why do we calculate ``n2p`` and ``p2n`` from ``N2P.varlam_np`` and ``N2P.varlam_pn`` in ``create_movie_arrays``, but we use ``N2P.driver`` in ``pp``???

## README file for ``PostProcessing.py``, ``support_functions.py``, and ``RunPostProcessing.ipynb``

* ``RunPostProcessing.ipynb`` installs ``PostProcessing.py`` and saves data for a given sterile neutrino model while printing graphs and other useful information...
* Before you can run ``RunPostProcessing.ipynb``, you need to download and install the following packages if you haven't already:
    * \<package name\> = ``DES``, \<file name\> = ``Differential_Equation_Solver_new.py``, \<description\> = Code to solve ODEs
    * \<package name\> = ``N2P``, \<file name\> = ``N2P_calc.py``, \<description\> = Neutron-to-proton code
* To install a package, follow the following steps:
1. Navigate to the directory that your BasicCode folder is in. Make a folder there called \<package name\>, then make a folder within \<package name\> that is also named \<package name\>.
2. Download \<file name\> from my GitHub page at https://github.com/hannahrasmussen/package_name (insert package name of interest into the URL) and put it in the folder within the folder.
3. In the folder (not the folder within the folder), put a ``setup.py`` folder with the following code:
        from setuptools import setup
        setup(name='package name',
          version='0.1',
          description='description',
          url='https://github.com/hannahrasmussen/package_name',
          author='Hannah Rasmussen',
          author_email='hannahrasmussen17@gmail.com',
          license='MIT',
          packages=['package name'],
          zip_safe=False)
          
4. Now, go to the terminal and checkout your new folder (not the folder within a folder) using the ‘cd’ command. 
5. Once you’re there, type and enter ‘pip install -e .’ (including the period). That ‘-e’ extension means that updates you make to the code are automatically also installed, which is obviously what we want.
6. Now, to see if it worked, restart your jupyter notebook kernel in ``RunBasicCode.ipynb`` and try and do ‘from \<package name\> import \<file name\>’. If there are no errors, you can delete that line and should be good to run the code as is.

# ``RunPostProcessing.ipynb``

# ``PostProcessing.py``

## Imports

* ``numpy`` is a python package that allows for quick and easy computing using multi-dimensional arrays.
* ``matplotlib.pyplot`` is a python package that provides a wide variety of 2-dimensional plotting options. 
* ``numba`` is a python package designed to interface with ``numpy`` and make the code run significantly faster (at least one order of magnitude)
* ``os`` is a python package that...
* ``matplotlib``'s ``animaiton``...
* ``scipy.interpolate``...
* ``CollisionApprox`` is a package that contains ``new_Collision_approx.py``, which approximates neutrino-electron/positron collisions. It calculates the "true" collision model from ``new_collisions_interp_extrap_0210.py`` every 100 steps, creates an approximatino of this calculation, and uses that approximation to model the neutrino-electron/positron collisions for the next 100 steps. See the README files in my CollisionApprox repo and in my nu_e_coll repo on GitHub for more documentation on these codes.
* ``support_functions``...
* ``N2P``...

## Physical Constants

* ``hbar``: Reduced Planck's constant, $\hbar = 6.58212 \times 10^{-22} \text{ MeV} \bullet \text{s}$
* ``mpi_neutral``: Neutral pion mass, $m_{\pi^0} = 135 \text{ MeV}$

## Equilibria calculations
Originally, we used scale factor, temperature, and $f$ arrays from *our* models to calculate lepton interconversion rates and the Hubble rate, but since the decaying sterile neutrinos affect the time temperature relation, this doesn't work, so we calculate these rates independent of anything in our model, outside of any graphing or movie-making functions so the same data is always used for comparison to our models.

### ``f_nu``
Neutrino ccupation fraction when neutrinos are in thermal equilibrium.
* **Imports**
    * ``E``: Neutrino energy
    * ``Tcm``: Current comoving temperature
* **Outputs**
    * $f_{\nu} = \frac{1}{e^{E/T_{\rm cm}}+1}$

* ``eq_a_arr``: Array of scale factors in the standard cosmological model, ``Luke_a.npy``
* ``eq_T_arr``: Array of plasma temperatures in the standard cosmological model, ``Luke_T.npy``
* ``eq_eta_arr``: Array of chemical potentials over scale factor in the standard cosmological model, ``Luke_eta.npy``
* ``eq_e_arr``: Array of epsilon values to be used to calculate ``eq_f_arr``.
* ``eq_f_arr``: Array of neutrino occupation fractions recalculated for each scale factor in ``eq_a_arr`` when calculating ``n2p_st`` and ``p2n_st``.
* ``cs_eta``: Cubic spline function of ``eq_eta_arr`` as a function of ``eq_T_arr``, such that a chemical potential is returned when a plasma temperature is sent to the function.
* ``cs_Tcm``: Cubic spline function of 1/``eq_a_arr`` as a function of ``eq_T_arr``, such that a comoving temperature is returned when a plasma temperature is sent to the function.
* ``n2p_st``: The neutron-to-proton rate in the standard cosmological model as calculated by ``N2P.varlam_np``.
* ``p2n_st``: The proton-to-neutron rate in the standard cosmological model as calculated by ``N2P.varlam_np``.
* ``Hub_st``: The Hubble rate in the standard cosmological model, $ H = \frac{da}{dt}\frac{1}{a}$.
* ``dtda``: The derivative of time with respect to scale factor as calculated from the function ``sf.time_derivative``.

## Creating movie arrays

### ``create_movie_arrays``
Creates arrays to be used in the rest of the movie and graph-making functions. 
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
* **Calculations**
    * ``eq_eta``: Array of chemical potentials calculated from ``cs_eta`` as a function of ``T_tot``.
    * ``max_e_len``: Maximum length of any of the epsilon arrays in the matrix ``e_tot``.
* **Outputs**
    * ``a_tot``: Array of scale factors loaded from ``folder/data``
    * ``e_tot``: Matrix of epsilon values loaded from ``folder/data``
    * ``f_tot``: Matrix of neutrino occupation fractions loaded from ``folder/data``
    * ``T_tot``: Array of plasma temperatures loaded from ``folder/data``
    * ``t_tot``: Array of time in units of MeV$^{-1}$ loaded from ``folder/data``
    * ``d_tot``: Matrix of derivatives (including $df/da(\epsilon)$, $dQ/da$, $dT/da$, $dt/da$) over scale factor (used to make derivative graph later).
    * ``c_tot``: Matrix of change in occupation fractions due to $\nu-e$ and $\nu-\nu$ collisions over scale factor.
    * ``n2p``: The neutron-to-proton rate in our model as calculated by ``N2P.varlam_np``.
    * ``n2p_st``: The neutron-to-proton rate in the standard cosmological model.
    * ``p2n``: The proton-to-neutron rate in our model as calculated by ``N2P.varlam_np``.
    * ``p2n_st``: The proton-to-neutron rate in the standard cosmological model.
    * ``Hub``: The Hubble rate in our model, $ H = \frac{da}{dt}\frac{1}{a}$, where $dt/da$ comes from ``d[-1]``.
    * ``Hub_st``: The Hubble rate in the standard cosmological model, $ H = \frac{da}{dt}\frac{1}{a}$.

## Graph-making functions

### ``check_int``
* **Imports**
    * ``a_arr``: Array of scale factors
    * ``e_mat``: Array of epsilon arrays, the epsilon array evolves over ``a_arr``
    * ``f_mat``: Array of neutrino occupation arrays, where each array is a function of the corresponding epsilon array in ``e_mat``
* **Outputs**
    * ``output``: First, this function calls ``N2P.set_arrs`` which is designed to shorten given $\epsilon$ and $f$ arrays to not include "empty" boxes at the end that weren't used in original calculations. Then, it integrates over $\epsilon/a$, to get an accurate integral over momentum. Ultimately, the integral over the momentum array with respect to $\epsilon$ will decrease steadily with the increase of scale factor, but will suddenly spike when there is a new $\epsilon$ array being used because the spacing in the integral is different I guess?? Ultimately, this simply acts as an indicator of where neutron to proton rates might have little jumps (that turn into huge spikes in cubic spline) since these rates integrate over ``p_arr``$^2$.
    
### ``pp``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
* **Calculations**
    * ``folder``: The folder that contains the results of interest, takes the form ``../../{ms}-{mixangle}-FullTestNew/{ms}-{mixangle}-FullTestNew``
    * ``data``: The .npz file that contains the result arrays, takes the form ``folder/mass_{}_mix_{:.4}.npz``
    * ``a_arr``: Array of scale factors from ``folder/data``
    * ``e_mat``: Matrix of epsilon values from ``folder/data``
    * ``f_mat``: Matrix of neutrino occupation fractions from ``folder/data``
    * ``T_arr``: Array of plasma temperatures from ``folder/data``
    * ``t_arr``: Array of times from ``folder/data``
    * ``N_eff``: N-effective value as calculated by ``N2P.N_eff``
    * ``dilution_factor``: Dilution factor as calculated by ``N2P.F``
    * ``spb_array``: Array of entropy-per-baryon values from ``N2P.spb``, depends solely on scale factor and plasma temperature
    * ``n2p``: The neutron-to-proton rate in our model as calculated by ``N2P.driver``.
    * ``p2n``: The proton-to-neutron rate in our model as calculated by ``N2P.driver``.
    * ``Hubble``: The Hubble rate in our model as calculated by ``N2P.driver``.
    * ``std_cosmo_spb``: Array of entropy-per-baryon values in the standard cosmological model (entropy-per-baryon is a constant $5.9 \times 10^{-9} k_B$).
    * ``t_YnYp``: Array of times that are the independent variable for ``Y_arr`` (I think it comes from a cubic spline an thus isn't just ``t_arr``).
    * ``Y_arr``: Contains three arrays, $Y_n$, $Y_p$, and $Y_n+Y_p$, that are functions of ``t_YnYp``. 
* **Outputs**
    * A neutron-to-proton, proton-to-neutron, and Hubble rate over temp graph
    * A graph comparing plasma temperature over comoving temperature in our model, to plasma temperature over comoving temperature in the standard cosmological model, to a line of slope -1 that represents comoving temperature over comoving temperature.
    * A graph comparing the evolution of the entropy per baryon in our model to the evolution of the entropy per baryon in the standard cosmological model.
    * A graph comparing the relative proton and neutron abundances over time.
    
### ``inspect_graphs``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
* **Calculations**
    * ``folder``: The folder that contains the results of interest, takes the form ``../../{ms}-{mixangle}-FullTestNew/{ms}-{mixangle}-FullTestNew``
    * ``data``: The .npz file that contains the result arrays, takes the form ``folder/mass_{}_mix_{:.4}.npz``
    * ``a_arr``: Array of scale factors from ``folder/data``
    * ``e_mat``: Matrix of epsilon values from ``folder/data``
    * ``f_mat``: Matrix of neutrino occupation fractions from ``folder/data``
    * ``T_arr``: Array of plasma temperatures from ``folder/data``
    * ``t_arr``: Array of times from ``folder/data``
    * ``n2p``: The neutron-to-proton rate in our model as calculated by ``N2P.driver``.
    * ``p2n``: The proton-to-neutron rate in our model as calculated by ``N2P.driver``.
    * ``Hubble``: The Hubble rate in our model as calculated by ``N2P.driver``.
    * ``j``: Begins at $10^{-10}$, and increases by $10^{-10}$ every time it is used. Ultimately used to make sure that ``t_arr`` is strictly increasing, for some reason there are some cases where ``t_arr`` repeats, and in order to make a cubic spline the independent variable must be strictly increasing.
    * ``t_YnYp``: Array of times that are the independent variable for ``Y_arr`` (I think it comes from a cubic spline an thus isn't just ``t_arr``).
    * ``results_YnYp``: Contains three arrays, $Y_n$, $Y_p$, and $Y_n+Y_p$, that are functions of ``t_YnYp``.
    * ``cs_n2p``: Cubic splined function that takes time (in terms of seconds) and returns neutron to proton rate
    * ``cs_p2n``: Cubic splined function that takes time (in terms of seconds) and returns proton to neutron rate
    * ``t_cs``: An array of time (in units of seconds) that will be used to make the arrays ``n2p_cs`` and ``p2n_cs``
    * ``n2p_cs``: A cubic splined array of neutron to proton rates as calculated by ``cs_n2p``, and a function of ``t_cs``
    * ``p2n_cs``: A cubic splined array of proton to neutron rates as calculated by ``cs_n2p``, and a function of ``t_cs``
    * ``nue_n``: Rate of the reaction $\nu + n \to p + e^-$ calculated ``a_arr``, ``e_mat``, ``f_mat``, and ``T_arr``
    * ``pos_n``: Rate of the reaction $\e^+ + n \to \bar{nu}_e + p$ calculated ``a_arr``, ``e_mat``, ``f_mat``, and ``T_arr``
    * ``n``: Rate of the reaction $n \to p + e^- + \bar{\nu}_e$ calculated ``a_arr``, ``e_mat``, ``f_mat``, and ``T_arr``
    * ``p_elec``: Rate of the reaction $p + e^- \to \nu + n $ calculated ``a_arr``, ``e_mat``, ``f_mat``, and ``T_arr``
    * ``anue_p``: Rate of the reaction $\bar{nu}_e + p \to \e^+ + n $ calculated ``a_arr``, ``e_mat``, ``f_mat``, and ``T_arr``
    * ``anue_elec_p``: Rate of the reaction $p + e + \bar{\nu}_e \to n$ calculated ``a_arr``, ``e_mat``, ``f_mat``, and ``T_arr``
    * ``integral``: Integral over ``p_arr``/``e_arr`` integrated over ``e_arr``, as calculated by ``check_int``
* **Outputs**
    * A graph comparing the relative proton and neutron abundances over time.
    * A "zoomed-in" version of the graph comparing the relative proton and neutron abundances over time, designed to zoom in on the area where $Y_n$ and $Y_p$ "blow up".
    * A graph of the neutron-to-proton rate, the proton-to-neutron rate, and the Hubble rate.
    * A graph of the neutron-to-proton rate, the proton-to-neutron rate, and the Hubble rate, but cubic splined! $Y_n$ and $Y_p$ are calculated from these cubic splined versions of the rates, and we can see that they blow up where there are little "bumps" in the normal n2p and p2n graph, thus causing the $Y_n$ and $Y_p$ graphs to "blow up" at these points.
    * A graph breaking down the six lepton reaction rates over *temperature* (why temperature and not time?).
    * A graph of the integral over ``p_arr``/``e_arr`` integrated over ``e_arr``, graphed over plasma temperature. Designed to highlight where the $\epsilon$ array changes. Unsurpirisingly, a lot of these spots where ``e_arr`` changes is where ``n2p`` and ``p2n`` have small bumps and aren't totally smooth.
    
### ``graph_f``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
    * ``save``: A boolean that determines if the graph will be saved; defaults to ``False`` but can be overwritten to be ``True``.
* **Outputs**
    * Returns a graph that compares the final neutrino occupation fraction $f(\epsilon)$ in our model to the final neutrino occupation fraction in the standard cosmological model.
    
### ``graph_df``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
    * ``save``: A boolean that determines if the graph will be saved; defaults to ``False`` but can be overwritten to be ``True``.
* **Outputs**
    * Returns a graph of the derivative of $f(\epsilon)$ in our model at the end of the code, so honestly should be an empty graph if ``basic_code.py`` ran as far as it should have as at the end of the code the neutrinos should be completely decoupled and thus $f$ shouldn;'t be changing any more.
    
### ``graph_TTcm``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
    * ``save``: A boolean that determines if the graph will be saved; defaults to ``False`` but can be overwritten to be ``True``.
* **Outputs**
    * Returns a graph comparing our model's plasma temperature $T$ over comoving temperature $T_{\rm cm}$ to the standard cosmological model plasma temperature over comoving temperature (hmm... should that be specifically std cosmological model comoving temp since the time temp relation is changed??), with a line with slope of -1 (representing $T_{\rm cm}$ over $T_{\rm cm}$) for comparison to both.
    
### ``graph_n2p``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
    * ``save``: A boolean that determines if the graph will be saved; defaults to ``False`` but can be overwritten to be ``True``.
* **Outputs**
    * Returns a graph of neutron to proton and proton to neutron rates divided by the Hubble rate, with the Hubble rate also included as a dashed horizontal line.
    
### `make_graphs``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
    * ``save``: A boolean that determines if the four graphs will be saved; defaults to ``False`` but can be overwritten to be ``True``.
* **Outputs**
    * Calls ``graph_f``, ``graph_df``, ``graph_TTcm``, and ``graph_n2p`` to make the four graphs in those respective functions.

## Movie-making functions

### ``movie_f``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
* **Outputs**
    * Returns a movie that compares how neutrino occupation fraction in our model changes over time to how neutrino occupation fraction in the standard cosmological model changes over time.
    
### ``movie_df``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
* **Outputs**
    * Returns a movie that shows the rate of change of $f$ and its componenents. The solid black line represents additions to $f$ from sterile neutrino decay products, the dashed black line represents additions to $f$ from collisions, and the dashed red line represents subtractions from $f$ from collisions.
    
### ``graph_TTcm``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
* **Outputs**
    * Returns a movie comparing our model's plasma temperature $T$ over comoving temperature $T_{\rm cm}$ to the standard cosmological model plasma temperature over comoving temperature (hmm... should that be specifically std cosmological model comoving temp since the time temp relation is changed??), with a line with slope of -1 (representing $T_{\rm cm}$ over $T_{\rm cm}$) for comparison to both.
    
### ``graph_n2p``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
* **Outputs**
    * Returns a movie of neutron to proton and proton to neutron rates divided by the Hubble rate, with the Hubble rate also included as a dashed horizontal line.
    
### `make_movies``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
* **Outputs**
    * Calls ``movie_f``, ``movie_df``, ``movie_TTcm``, and ``movie_n2p`` to make the four movies in those respective functions.
    
### ``make_movie_3``
* **Imports**
    * ``ms``: Sterile neutrino mass
    * ``mixangle``: Sterile neutrino mixing angle
* **Outputs**
    * Makes a total of three movies in one, with evolution of $f$ (compared to std model $f$) in the top left corner, the derivative of $f$ in the bottom left corner, and with $T/T_{\rm cm}$ at right.

# ``support_functions.py``

## Imports

* ``numpy`` is a python package that allows for quick and easy computing using multi-dimensional arrays.
* ``matplotlib.pyplot`` is a python package that provides a wide variety of 2-dimensional plotting options. 
* ``numba`` is a python package designed to interface with ``numpy`` and make the code run significantly faster (at least one order of magnitude)
* ``time`` is a python package that...
* ``DES`` is an ODE solver package that contains ``Differential_Equation_Solver_new.py``, which solves differential equations using a Fourth-Order Runge Kutta routine with an adaptive stepsize. See the README file in my DES repo on GitHub for more documentation on this code.
* ``nu_nu_coll`` is a package that contains ``nu_nu_collisions.py``, which models neutrino-neutrino collisions at every step. See the README file in my nu_nu_coll repo on GitHub for more documentation on this code.
* ``CollisionApprox`` is a package that contains ``new_Collision_approx.py``, which approximates neutrino-electron/positron collisions. It calculates the "true" collision model from ``new_collisions_interp_extrap_0210.py`` every 100 steps, creates an approximatino of this calculation, and uses that approximation to model the neutrino-electron/positron collisions for the next 100 steps. See the README files in my CollisionApprox repo and in my nu_e_coll repo on GitHub for more documentation on these codes.
* ``Interpolate`` is a package that contains ``interp.py``, which ... See the README file in my Interpolate repo on GitHub for more documentation on this code.

## Physical Constants

* ``alpha``: The unitless fine structure constant, $\alpha = 1/137$ 
* ``D``: Parameter that describes the dilution of the sterile neutrinos from the time they decoupled, $$ \begin{aligned} D &= \left(\left. \frac{T}{T_{\rm cm}}\right\vert_{\rm \nu_s \text{-} {\rm dec}} \cdot \left. \frac{T_{\rm cm}}{T} \right\vert_{\rm wd} \right)^3 \approx \frac{1}{1.79^3} \\ &= \left(\frac{g_{\rm wd}}{g_{{\rm \nu_s \text{-} dec}}}\right) \approx \left(\frac{10.75}{61.75}\right) \end{aligned} $$
* ``dtda_part2``:  Used in driver functions but can be calculated out here, $\frac{2\pi}{3} $ 
* ``Enumax``: Maximum energy of the muon and electron neutrino from the decay of the muon in the rest frame of the muon, $E_{\nu \rm max} = \frac{m_{\mu}}{2}(1-\epsilon_e^2)$
* ``eps_e``: Constant used in calculations of the derivative of decays 3 and 4 when $\epsilon = 0$, $\epsilon_e = \frac{m_e}{m_{\mu}}$
* ``f_pi``: Pion decay constant, $f_{\pi} = 131 \text{ MeV}$
* ``Gf``: Fermi constant, $G_F = 1.166 \times 10 ^{-11} \text{ MeV}^{-2}$
* ``me``: Electron mass, $m_e = 0.511 \text{ MeV}$
* ``mpi_neutral``: Neutral pion mass, $m_{\pi^0} = 135 \text{ MeV}$
* ``mpi_charged``: Charged pion mass, $m_{\pi^{\pm}} = 139.569 \text{ MeV}$
* ``mPL``: Planck mass, $m_{\rm Pl} = 1.124 \times 10^{22} \text{ MeV}$
* ``mu``: Muon mass, $m_{\mu} = 105.661 \text{ MeV}$

## Model-specific parameters

* ``n``: Number of steps for Gauss-Laguerre and Gauss-Legendre quadrature, max is about 150 before it stops working well
* ``f_TINY`` = 1e-20, used in the ``find_breaks`` function
* ``f_MINI`` = 1e-25, used in the ``find_breaks`` function
* ``f_SMALL`` = 1e-30, used in several functions
* ``f_BUFFER`` = 1e-40, used in several functions
* ``MIN_eps_BUFFER`` = 12, used in several functions
* ``a_MAXMULT`` = 2, value of $a$ at which driver will stop running 
* Gauss-Laguerre quadrature: A mathematical tool to evaluate any integral from $[0,\infty)$ using Laguerre polynomials. You send $n$ to the function and it returns the $n$ roots of the $n^{\rm th}$ Laguerre polynomial. These $n$ roots are ``x_values``. Each root has a corresponding weight in ``w_values``. By evaluating $\sum_{i=0}^{n-1} w_i * e^{-x_i}f(x_i)$, an approximation for $\int_0^{\infty} f(x) dx$ is returned.
* ``x_values``: The $n$ roots of the $n^{\rm th}$ Laguerre polynomial
* ``w_values``: The weights that correspond to ``x_values``
* Gauss-Legendre quadrature: A mathematical tool to evaluate any integral from $[-1,1]$ using Legendre polynomials. You send $n$ to the function and it returns the $n$ roots of the $n^{\rm th}$ Legendre polynomial. These n roots are ``x_valuese``. Each root has a corresponding weight in ``w_valuese``. By evaluating $\sum_{i=1}^n w_i f(x_i)$, an approximation for $\int_{-1}^1 f(x) dx$ is returned. Note that this can be molded to calculate any definite integral, $\int_a^b f(x) dx$.
* ``x_valuese``: The $n$ roots of the $n^{\rm th}$ Legendre polynomial
* ``w_valuese``: The weights that correspond to ``x_valuese``

## Integral-related functions

### ``I1``  - the integrand in $\rho_{e^{\pm}}$
The energy density of electrons and positrons, $\rho_{e^{\pm}}$, is $$ \rho_{e^{\pm}} = \frac{2T^4}{\pi^2}\int_0^\infty \text{I1 }d\xi = \frac{2T^4}{\pi^2}\int_0^\infty\frac{\xi^2\sqrt{\xi^2+x^2}}{e^{\sqrt{\xi^2+x^2}}+1}d\xi. $$ This function, however, only computes the *integrand* portion of that equation. In addition, what is returned by this function has an additional $e^{\xi}$ as necessitated by the mathematical tool we use to evaluate this integral, Gauss-Laguerre quadrature (see above). The coefficient, $\left(2T^4 \right)/ \pi^2$, is ultimately embedded into $dT/da$ below.
* **Inputs**: 
    * ``xi``: A dimensionless expression of $\xi = p/T$, and our dummy variable of integration. Notice that $\rho_{e^{\pm}}$ is entirely a function of temperature (in thermal equilibrium) as momentum is integrated out.
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which $\rho_{e^{\pm}}$ is being calculated.
* **Outputs**: 
    * ``E_den``: The integrand at the given $\xi$ and $x$ value: $$ \frac{e^{\xi} \xi^2\sqrt{\xi^2+x^2}}{e^{\sqrt{\xi^2+x^2}}+1} $$
    
### ``I2`` - the integrand in $P_{e^{\pm}}$
The pressure of electrons and positrons, $P_{e^{\pm}}$, is $$ P_{e^{\pm}} = \frac{2T^4}{3\pi^2}\int_0^\infty \text{I2 } d\xi = \frac{2T^4}{3\pi^2}\int_0^\infty\frac{e^{\xi} \xi^4}{\sqrt{\xi^2+x^2} \left(e^{\sqrt{\xi^2+x^2}}+1 \right)} d\xi. $$ This function, however, only computes the *integrand* portion of that equation. In addition, what is returned by this function has an additional $e^{\xi}$ as necessitated by the mathematical tool we use to evaluate this integral, Gauss-Laguerre quadrature (see above). The coefficient, $\left(2T^4 \right)/ \left(3\pi^2 \right)$, is ultimately embedded into $dT/da$ below.
* **Inputs**: 
    * ``xi``: A dimensionless expression of $\xi = p/T$, and our dummy variable of integration. Notice that $P_{e^{\pm}}$ is entirely a function of temperature (in thermal equilibrium) as momentum is integrated out.
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which $P_{e^{\pm}}$ is being calculated.
* **Outputs**: 
    * ``Pressure``: The integrand at the given $\xi$ and $x$ value: $$ \frac{e^{\xi} \xi^4 }{\sqrt{\xi^2+x^2} \left(e^{\sqrt{\xi^2+x^2}}+1 \right)} $$
    
### ``dI1`` - the integrand in $d\rho_{e^{\pm}}/dT$
The derivative of the energy density of electrons and positrons, $d\rho_{e^{\pm}}/dT$, is $$ \frac{d\rho_{e^{\pm}}}{dT} = \frac{8T^3}{\pi^2}\int_0^\infty \text{I1 }d\xi + \frac{2Tm_e^2}{\pi^2} \int_0^{\infty} \text{dI1 } d\xi = \frac{8T^3}{\pi^2}\int_0^\infty\frac{\xi^2\sqrt{\xi^2+x^2}}{e^{\sqrt{\xi^2+x^2}}+1}d\xi + \frac{2Tm_e^2}{\pi^2} \int_0^{\infty} \frac{\sqrt{\xi^2 + x^2}}{e^{\sqrt{\xi^2 + x^2}}+1} d\xi . $$ 

Notice that we already have code for the integrand in the first term, ``I1``!! This function computes the *integrand* portion of the second term, ``dI1``. In addition, what is returned by this function has an additional $e^{\xi}$ as necessitated by the mathematical tool we use to evaluate this integral, Gauss-Laguerre quadrature (see above). The coefficients of both terms, $\left( 8T^3 \right) / \pi^2$ and $ \left( 2Tm_e^2 \right)/ \pi^2$ are both ultimately embedded into $dT/da$ below.
* **Inputs**: 
    * ``xi``: A dimensionless expression of $\xi = p/T$, and our dummy variable of integration. Notice that $d\rho_{e^{\pm}}$ is entirely a function of temperature (in thermal equilibrium) as momentum is integrated out.
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which $d\rho_{e^{\pm}}$ is being calculated.
* **Outputs**: 
    * ``dE_den``: The integrand at the given $\xi$ and $x$ value: $$\frac{e^{\xi} \sqrt{\xi^2 + x^2}}{e^{\sqrt{\xi^2 + x^2}}+1}$$
    
### ``dI2`` - the integrand in $dP_{e^{\pm}}/dT$
The derivative of the energy density of electrons and positrons, $d\rho_{e^{\pm}}/dT$, is $$ \frac{d\rho_{e^{\pm}}}{dT} = \frac{8T^3}{3\pi^2} \int_0^\infty \text{I2 }d\xi + \frac{2Tm_e^2}{3\pi^2} \int_0^{\infty} \text{dI2 } d\xi = \frac{8T^3}{3\pi^2} \int_0^\infty \frac{\xi^4}{\sqrt{\xi^2+x^2} \left(e^{\sqrt{\xi^2+x^2}}+1 \right)} d\xi + \frac{2Tm_e^2}{3\pi^2} \int_0^{\infty} \frac{3\xi^2}{\sqrt{\xi^2 + x^2} \left(e^{\sqrt{\xi^2 + x^2}}+1 \right)} d\xi . $$ 

Notice that we already have code for the integrand in the first term, ``I2``!! This function computes the *integrand* portion of the second term, ``dI2``. In addition, what is returned by this function has an additional $e^{\xi}$ as necessitated by the mathematical tool we use to evaluate this integral, Gauss-Laguerre quadrature (see above). The coefficients of both terms, $\left( 8T^3 \right)/\left(3\pi^2 \right)$ and $ \left(2Tm_e^2 \right)/\left(3\pi^2 \right)$ are both ultimately embedded into $dT/da$ below.
* **Inputs**: 
    * ``xi``: A dimensionless expression of $\xi = p/T$, and our dummy variable of integration. Notice that $dP_{e^{\pm}}$ is entirely a function of temperature (in thermal equilibrium) as momentum is integrated out.
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which $dP_{e^{\pm}}$ is being calculated.
* **Outputs**: 
    * ``dPressure``: The integrand at the given $\xi$ and $x$ value: $$ \frac{e^{\xi} 3\xi^2}{\sqrt{\xi^2 + x^2} \left(e^{\sqrt{\xi^2 + x^2}}+1 \right)} $$
    
### ``calculate_integral`` 
Uses a Gauss-Laguerre quadrature to integrate over one of the four integrands above, ``I1``, ``I2``, ``dI1``, or ``dI2``. Note that ``x_values``, as defined above, is the array of roots of the $n^{\rm th}$ Laguerre polynomial, while ``x`` is the adjusted temperature that is specific to each individual calculation. I couldn't really think of a more appropriate way to name either of them, but it is really only in this one particular instance that it's confusing.
* **Inputs**: 
    * ``I``: The function that returns the integrand over which we will be performing the integral. 
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which the integral, ``I``, is being calculated.
* **Outputs**: 
    * ``integral``: A summation over the integrand ``I`` using Gauss-Laguerre quadrature.
    
### ``trapezoid``
Performs a summation over a given array using trapezoid rule. Good for relatively simple integrals over smooth curves.
* **Inputs**: 
    * ``array``: Array of values that we will be integrating over, i.e., an array of some variable that depends on ``x``
    * ``dx``: Spaces between bins of a linearly spaced independent variable array
* **Outputs**: 
    * ``total``: The result of the summation: $$ \sum_{i=0}^{len(\text{array})-1} \frac{ dx * (\text{array}[i]+\text{array}[i+1])}{2} $$

## Decay rates

### ``rate1``
Calculates the decay rate of decay 1, $\nu_s \to \nu + \gamma$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 1, $$\Gamma_1 = \frac{9 G_F^2 \alpha m_s^5 \sin^2(\theta)}{512 \pi^4}$$

### ``rate2``
Calculates the decay rate of decay 2, $\nu_s \to \nu + \pi^0$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 2, $$\Gamma_2 = \frac{G_F^2 f_{\pi}^2}{16 \pi} m_s (m_s^2-m_{\pi^0}^2) \sin^2(\theta)$$
    
### ``rate3``
Calculates the decay rate of decay 3, $\nu_s \to \pi^{\pm} + e^{\mp}$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 3, there is an extra 2 coefficient out front because $\nu_s$ can decay into either $\pi^+$ and $e^-$ OR $\pi^-$ and $e^+$, $$\Gamma_3 = 2 \frac{G_F^2 f_{\pi}^2}{16 \pi} m_s \sqrt{(m_s^2 - (m_{\pi^0}+m_e)^2)*(m_s^2 - (m_{\pi^0}-m_e)^2)} \sin^2(\theta)$$ 

### ``rate4``
Calculates the decay rate of decay 4, $\nu_s \to \pi^{\pm} + \mu^{\mp}$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 4, there is an extra 2 coefficient out front because $\nu_s$ can decay into either $\pi^+$ and $\mu^-$ OR $\pi^-$ and $\mu^+$, $$\Gamma_4 = 2 \frac{G_F^2 f_{\pi}^2}{16 \pi} m_s \sqrt{(m_s^2 - (m_{\pi^{\pm}}+m_{\mu})^2)*(m_s^2 - (m_{\pi^{\pm}}-m_{\mu})^2)} \sin^2(\theta)$$ 
    
### ``ts``
Calculates the lifetime of the sterile neutrino (in units of MeV$^{-1}$) when only decay processes 1-4 are allowed. 
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``tau``: The overall lifetime of the sterile neutrino at the given mass and mixing angle in units of MeV$^{-1}$, $$\tau = \frac{1}{\Gamma_1 + \Gamma_2 + \Gamma_3 + \Gamma_4}$$
    
### ``ns``
Function that defines $n_s$, the number density of sterile neutrinos:
* **Inputs**:
    * ``Tcm``: Comoving temperature ($1/a$)
    * ``t``: The time in units of MeV$^{-1}$
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**:
    * ``n_s``: The number density of the sterile neutrino at the given time and comoving temperature (since $T_{\rm cm}$ describes the sterile neutrino population), $$ n_s = \frac{3D \zeta(3)}{2 \pi^2} T_{\rm cm}^3 e^{-t/\tau_s}$$

## Relativity functions

### ``diracdelta`` 
In the decay of type $\nu_s \to B+C$, this function describes the monoenergtic probability distribution of the energy of particle $B$ in the rest frame of $\nu_s$
* **Inputs**: 
    * ``E``: Energy of current energy bin that we're looking at, always sends ``eps*Tcm``
    * ``EI``: Energy of particle $B$ in the rest frame of the parent particle
    * ``i``: current index, that is, ``E`` = ``E_arr[i]``
    * ``E_arr``: the energy array, equivalent to ``e_array*Tcm``
* **Calculations**: 
    * Remember that a dirac-delta function, when integrated over, returns one. So the goal is to assign values that will give one when integrated over. 
    * The first set of if statements determines what the boxsizes to the left and to the right of the current bin are. If the current index is 0, then both boxsizes are set to be equivalent to the boxsize to the right of the zeroth index. If the current index is -1, then both boxsizes are set to be equivalent to the boxsize to the left of the final index. Otherwise, both boxsizes are simply calculated by taking the difference between energy values at the current index and those preceding and following it.
    * The next set of if statements is a bit more complex, and determines our output. 
        * In the first case, 
        * if $E_I - (0.6 * \text{bxR}) \leq E \leq E_I - (0.4 * \text{bxR})$:
            * $x = E_I - E - (0.5 * \text{bxR})$
            * $A = 0.1 * \text{bxR}$
            * return $ \displaystyle \frac{2}{\text{bxL}+\text{bxR}} * \bigg(0.5 + \frac{0.75}{A^3} * \Big(\frac{x^3}{3} - xA^2 \Big) \bigg)$
        * In the second case, $E$ 
        * elif $E_I - (0.4 * \text{bxR}) \leq E \leq E_I + (0.4 * \text{bxL})$:
            * return $\displaystyle \frac{2}{\text{bxL} + \text{bxR}}$
        * In the third case,
        * elif $E_I + (0.4 * \text{bxL}) \leq E \leq E_I + (0.6 * \text{bxL})$:
            * $x = E_I - E + (0.5 * \text{bxL})$
            * $A = 0.1 * \text{bxL}$
            * return $ \displaystyle \frac{2}{\text{bxL}+\text{bxR}} * \bigg(0.5 - \frac{0.75}{A^3} * \Big(\frac{x^3}{3} - xA^2 \Big) \bigg)$
        * In the final case, the energy of the current bin, $E$, is not within half a boxsize of $E_I$, so we return 0.
        * else:
            * return 0
* **Outputs**: 
    * If $E$ is within half a boxsize of $E_I$, then $E$ is in the energy bin that recieves all of the monoenergetic decay products of energy $E_I$, output = $\delta(E-E_I)$
        
### ``diracdelta2``  
In the decay of type $\nu_s \to A \to B + C$, this function describes the energy distribution in the Home Frame ($\nu_s$ rest frame) of a particle that is monoenergetic in the Other Frame (particle $A$'s rest frame) (describes energy distribution of particles $B$ or $C$ in this scenario in the rest frame of $\nu_s$)
* **Inputs**: 
    * ``E``: Energy of current energy bin that we're looking at, always sends ``i*boxsize``
    * ``EBmin``: Maximum energy of particle $B$ in the Home Frame
    * ``EBmax``: Minimum energy of particle $B$ in the Home Frame
    * ``E_B``: Energy of particle $B$ in the Other Frame, i.e, energy of particle $B$ in the rest frame of particle $A$ from the monoenergetic decay of $A \to B+C$
    * ``v``: The speed of particle $A$'s rest frame relative to the sterile neutrino rest frame, $v = p_A/E_A$
    * ``gamma``: The Lorentz factor of particle $A$'s rest frame relative to the sterile neutrino rest frame, $\gamma = E_A/m_A $
* **Outputs**: 
    * ``tophat``: The probability of making a particle with energy $E$ in the rest frame of the sterile neutrino, $\dfrac{1}{2\gamma v E_B} \cases{1 \text{ if } E_{\rm min}<E<E_{\rm max} \\ 0 \text{ otherwise}}$
    
### ``EB`` 
In the decay of type $A \to B+C$ (where $B$ is monoenergetic), returns the energy of $B$ in the rest frame of $A$.
* **Inputs**: 
    * ``mA``: Mass of particle $A$ (decaying particle)
    * ``mB``: Mass of particle $B$ (decay product whose energy we're interested in)
    * ``mC``: Mass of particle $C$ (other decay product)
* **Outputs**: 
    * ``E_B``: Energy of particle $B$ in the rest frame of particle $A$, $$E_B = \frac{m_A^2 + m_B^2 - m_C^2}{2m_A}$$
    
### ``Gammamua``
Integral over differential decay rate of the muon in decay types III and IV in the rest frame of the muon. Lorentz transformed further in other parts of code to return to rest frame of the sterile neutrino, but this part of the code exclusively works with the three-product decay of the muon in the rest frame of the muon. The primes are there to remind us that this needs to be further transformed to be in the Home Frame.

So the purpose of this code is to take an neutrino energy bin in the rest frame of the mother particle of the muon, $E$, figure out the minimum and maximum neutrino energies ($a$ and $b$, respectively) that could Lorentz transform to $E$ from the rest frame of the muon. 
* **Inputs**: 
    * ``a``: Given that the current neutrino energy bin is $E$, the minimum possible neutrino energy in the rest frame of the muon that could Lorentz transform to $E$ in the rest frame of the mother particle of the muon is $a = \frac{E}{\gamma_{\mu}(1+v_{\mu})}$. 
    * ``b``: Given that the current neutrino energy bin is $E$, the maximum possible neutrino energy in the rest frame of the muon that could Lorentz transform to $E$ in the rest frame of the mother particle of the muon is $ \frac{E}{\gamma_{\mu}(1-v_{\mu})}$. However, the neutrinos from this decay process cannot have a energy greater than ``Enumax`` where $E_{\nu \rm max} = \frac{m_{\mu}}{2}(1-\epsilon_e^2)$, so we set $b = {\rm min}(E_{\rm max}', \frac{E}{\gamma_{\mu}(1-v_{\mu})})$.
* **Outputs**: 
    * ``Gam_mua``: The integral over the differential decay rate $\frac{d\Gamma_{\mu}'}{dE_{\nu}'}\frac{dE_{\nu}'}{E_{\nu}'}$ in the rest frame of the muon, from the smallest possible energy of the neutrino that could Lorentz transform to $E$ in the mother particle rest frame, $a$, to the greatest possible energy of the neutrino that could Lorentz transform to $E$ in the mother particle rest frame, $b$, $$ \begin{aligned} \Gamma_{\mu_a}' &= \int^{b}_a \frac{d\Gamma_{\mu}'}{dE_{\nu}'} \frac{dE_{\nu}'}{E_{\nu}'} \\ &= \frac{8 G_F m_{\mu}^2}{16\pi^3} \int^b_a \frac{E_{\nu}'}{1-\frac{2E_{\nu}'}{m_{\mu}}} \left( 1 - \frac{2E_{\nu}'}{m_{\mu}} - \frac{m_e^2}{m_{\mu}^2} \right)^2 dE_{\nu}'  \\ &= \frac{-8 G_F m_{\mu}^2}{16\pi^3} \left. \left( \frac{\frac{1}{4} m_e^4 m_{\mu} \log{\left(|{2 E_{\nu}' - m_{\mu}}|\right)} + \frac{1}{6} E_{\nu}' \left(3 m_e^4 + 6E_{\nu}' m_e^2 m_{\mu} + m_{\mu}^2 E_{\nu}' (4E_{\nu}' - 3m_{\mu}) \right)}{m_{\mu}^3} \right) \right\vert_a^b \end{aligned} $$


### ``Gammamub``
Calculates integral over differential decay rate of the muon in decay types III and IV in the rest frame of the muon, but without the $E_{\nu}'$ in the denominator to isolate just the muon decay rate part of the integral. Lorentz transformed further in other parts of code to return to rest frame of the sterile neutrino, but this part of the code exclusively works with the three-product decay of the muon in the rest frame of the muon. The primes are there to remind us that this needs to be further transformed to be in the Home Frame.   
* **Inputs**: 
    * None
* **Outputs**: 
    * ``Gam_mub``: The integral over the differential decay rate $\frac{d\Gamma_{\mu}'}{dE_{\nu}'}dE_{\nu}'$ from the smallest possible energy of the neutrino in the muon rest frame, 0, to the greatest possible energy of the neutrino in the muon rest frame, $E_{\rm max}'$, $$ \begin{aligned} \Gamma_{\mu_b}' &= \int^{E_{\rm max}'}_0 \frac{d\Gamma_{\mu}'}{dE_{\nu}'}dE_{\nu}' \\ &= \frac{8 G_F m_{\mu}^2}{16\pi^3} \int^{E_{\rm max}'}_0 \frac{(E_{\nu}')^2}{1-\frac{2E_{\nu}'}{m_{\mu}}} \left( 1 - \frac{2E_{\nu}'}{m_{\mu}} - \frac{m_e^2}{m_{\mu}^2} \right)^2 dE_{\nu}'  \\ &= \frac{-8 G_F m_{\mu}^2}{16\pi^3} \left. \left(\frac{3 m_e^4 m_{\mu}^2 \log{\left(|{2 E_{\nu}' - m_{\mu}}|\right)} + 6 m_e^4 E_{\nu}' (m_{\mu}+E_{\nu}') + 16 m_e^2 m_{\mu} {E_{\nu}'}^3 + 4 m_{\mu}^2 {E_{\nu}'}^3 (3{E_{\nu}'} - 2m_{\mu})}{24 m_{\mu}^3} \right) \right\vert_0^{E_{\rm max}'} \end{aligned}$$
    
### ``Gam_mub``
Integral over $\frac{d\Gamma_{\mu}'}{dE_{\nu}'}dE_{\nu}'$, it's constant and only needs to be calculated once

### ``u_integral``
Using Gauss-Legendre quadrature, this function calculates the integral over all possible muon energies in the rest frame of the pion (for Decay Type IV) to Lorentz transform the active neutrino spectra into this rest frame.
* **Inputs**: 
    * ``E_mumin``: Minimum energy of the muon from the pion decay in the rest frame of the sterile neutrino
    * ``E_mumax``: Maximum energy of the muon from the pion decay in the rest frame of the sterile neutrino
    * ``Eactive``: Current neutrino energy bin in the Home Frame, ``i*boxsize``, $E$
* **Calculations** 
    * ``Eu_array``: Array of muon energies in the rest frame of the sterile neutrino that is created in accordance with Gauss-Legendre quadrature
    * ``gammau``: The Lorentz factor of the pion's rest frame to the rest frame of the sterile neutrino at the current energy bin
    * ``pu``: The momentum of the muon in the sterile neutrino rest frame at the current energy bin
    * ``vu``: The speed of the pion's rest frame relative to the sterile neutrino's rest frame at the current energy bin
    * ``Gam_mua``: The integral over the muon differential decay rate, $\Gamma_{\mu_a}' = \int_a^b \frac{d\Gamma_{\mu}'}{dE_{\nu}'}\frac{dE_{\nu}'}{E_{\nu}'}$ where $a= E/(\gamma_{\mu}(1+v_{\mu}))$ and $b= {\rm min}(E_{\nu \rm max}, E/(\gamma_{\mu}(1-v_{\mu})))$.
* **Outputs**: 
    * ``integral``: Calculates the integral $$\int_{E_{\mu \rm min}}^{E_{\mu \rm max}} \frac{\Gamma_{\mu_a}'}{2\gamma_{\mu}v_{\mu}} dE_{\mu}$$

## More functions...

    
### ``C_round`` 
This function calls ``cI`` from ``nu_nu_collisions.py``, which returns the integral $$ \begin{aligned} I = \frac{G_F^2}{(2\pi^3)p_1^2} \Bigg\{  & \int_0^{p_1} dp_2 \left[ \int_0^{p_2} dp_3 FJ_1(p_1,p_2,p_3) + \int_{p_2}^{p_1} dp_3 FJ_2(p_1,p_2) + \int_{p_1}^{p_1+p_2} dp_3 FJ_3(p_1,p_2,p_3) \right] \Bigg. \\ \Bigg. &+ \int_{p_1}^{\infty} dp_2 \left[ \int_0^{p_1}dp_3 FJ_1 (p_1,p_2,p_3) + \int_{p_1}^{p_2} dp_3 FJ_2(p_2,p_1) + \int_{p_2}^{p_1+p_2} dp_3 FJ_3 (p_1,p_2,p_3) \right] \Bigg\} \end{aligned} $$ for a given momentum $p_1$.
* **Inputs**: 
    * ``j``: The index of the current momentum of interest ($p_1$ above).
    * ``f``: The current array of neutrino occupation fractions.
    * ``p``: The array of neutrino momenta.
* **Outputs**: 
    * ``c`` or 0: ``cI`` returns both ``c``, which subtracts $F_m$ from $F_p$, and ``FRS``, which adds $F_m$ and $F_p$. ``FRS`` is used to ensure any nonzero values in ``c`` are not the result of computational error. If the difference between $F_p$ and $F_m$ is not greater than $3 \times 10^{-15}$ the sum of $F_p$ and $F_m$, 0 is returned. Otherwise, ``c`` is returned.
    
### ``C_short`` 
This function calls ``C_round`` to calculate changes to $f$ from $\nu-\nu$ collisions, but it does so using three if statements designed to send only necessary fragments of $p$ and $f$ with the goal of saving time (hence, C "short"). The fragments are determined by ``k`` from the ``find_breaks`` function...
* **Inputs**: 
    * ``p``: Array of neutrino momenta
    * ``f``: Current array of neutrino occupation fraction
    * ``T``: Current plasma temperature
    * ``k``: A length three array that comes from the ``find_breaks`` function. Generally, ``k[0]`` is the index after the final ``f`` index where ``f``<$1 \times 10^{-20}$, ``k[1]`` is the index after the final ``f`` index where ``f``<$1 \times 10^{-25}$, and ``k[2]`` is the index after the final ``f`` index where ``f``<$1 \times 10^{-30}$. There are exceptions that make these k values bigger if they are too close to the indices that correspond to the energies of neutrinos from decays 1 and 2, see ``find_breaks`` info below. 
* **Outputs**: 
    * ``c``: An array of the change in neutrino occupation fraction due to $\nu-\nu$ collisions. A for loop is used to calculate this change for each box of ``p``. If ``i``, the current index of ``p``, is less than ``k[0]``, then only ``f[:k[1]]`` is sent to ``C_round`` to be used to calculate the change in $f$ due to $\nu-\nu$ collisions for this specific box. The idea is that boxes whose $f < 10^{-25}$ will not really contribute to the change of any box whose $f > 10^{-20}$. Similarly, if ``i`` is less than ``k[1]`` (but not less than ``k[0]``), then only ``f[:k[2]]`` is sent to ``C_round`` to be used to calculate the change in $f$ due to $\nu-\nu$ collisions for this specific box. The idea is that boxes whose $f < 10^{-30}$ will not really contribute to the change of any box whose $f > 10^{-25}$. If ``i``>``k[1]``, the entire $f$ array is sent to ``C_round``.
    
### ``find_breaks`` 
This function sets indices that will be used in ``C_short`` to shorten the time spent calculating $\nu-\nu$ collisions. It uses the contants ``f_TINY``, ``f_MINI``, ``f_SMALL``, and ``MIN_eps_BUFFER``, which are $1 \times 10^{-20}$, $1 \times 10^{-25}$, $1 \times 10^{-30}$, and 12, respectively. 
* **Inputs**: 
    * ``f``: The array of neutrino occupation fractions.
    * ``E2_index``: The index of the epsilon array (that corresponds to ``f``) that contains $E_2$, the energy of neurtino products from decay 2. 
    * ``E1_index``: The index of the epsilon array (that corresponds to ``f``) that contains $E_1$, the energy of neurtino products from decay 1. 
* **Calculations**
    * The if-else statments set ``k_0``, ``k_1``, and ``k_2`` to be the first index where ``f``<``f_TINY``, ``f``<``f_MINI``, and ``f``<``f_SMALL``, respectively. If ``f`` is never this small, each k value is simply set to ``len(f)-1``. 
    * There are then three for loops that go from ``k_0``, ``k_1``, and ``k_2`` to the end of ``f`` to check which values of ``f`` are greater than ``f_TINY``, ``f_MINI``, and ``f_SMALL``, respectively. Any time ``f`` is greater than the respective constant, the k value is set to equal this index PLUS ONE, such that ``k_0``, ``k_1``, and ``k_2`` end up marking the index AFTER the final place that ``f``>``f_TINY``, ``f``>``f_MINI``, and ``f``<``f_SMALL``. Ultimately, it should always be the case that ``k_0``<``k_1``<``k_2``.
    * Finally, we enter a set of nested for loops that are basically designed to add a "buffer" to a k-value if it is smaller than or close to being smaller than ``E2_index`` or ``E1_index``. This is to make sure that in trying to calculate the $\nu-\nu$ collisions more quickly by ignoring $f-values$ that are too small that we don't ignore *too* much by ignoring values around the result of decays 1 and 2 (which have the highest energy decay products). The outside for loop iterates three times to go through each of the three k values.
        * The first nested for loop iterates twice for the two indices of ``E_check``, which contains ``E2_index`` and ``E1_index``, and contains two if statements. The first states that if ``k_return[j]`` is between ``E_check[i]-MIN_eps_BUFFER`` and ``E_check[i]``, then ``k_return[j]`` = ``k_return[j]`` + 2``MIN_eps_BUFFER``. The second if statement states that if ``k_return[j]`` is between ``E_check[i]`` and ``E_check[i]`` + ``MIN_eps_BUFFER``, then ``k_return[j]`` = ``k_return[j]`` + ``MIN_eps_BUFFER``.
        * The second nested for loop goes through the remaining k values, and if a k value is less than a preceding k value + ``MIN_eps_BUFFER``, then the current k value is set to that preceding k value plus ``MIN_eps_BUFFER``: ``k_return[jj]`` = ``k_return[j]`` + ``MIN_eps_BUFFER``.
        * The outside for loop concludes with an if statement that sets any of the three k values to ``len(f)-1`` is they are greater than or equal to ``len(f)``.
* **Outputs**: 
    * ``k_return``: An array of length three that contains ``k_0``, ``k_1``, and ``k_2``.

# ODES 

Ok this part is probably going to be a work in progress, especially considering that I'm pretty sure a more recent version of this code exists somewhere that I need to get from Chad... but I'm pretty sure at this stage, the code is designed to go through driver for 100 steps, save the data, and then do it again to make a new model for $\nu-e$ collisions?

## ``make_collision_fn``
Decay 1: $$\nu_s \rightarrow \nu + \gamma$$ Home Frame is simply the rest frame of the sterile neutrino, no Other Frame is considered.

Decay 2: $$\nu_s \rightarrow \nu + \pi^0$$ Home Frame is simply the rest frame of the sterile neutrino, no Other Frame is considered.

Decay 3a: $$\nu_s \rightarrow \pi^{\pm} + e^{\mp} \rightarrow \nu_{\mu} + \mu^{\pm} $$ Home Frame is the rest frame of the sterile neutrino, Other Frame is the rest frame of the pion. We're interested in the muon neutrino produced by the decay of the pion.

Decay 3b: $$\nu_s \rightarrow \pi^{\pm} + e^{\mp} \rightarrow \nu_{\mu} + \mu^{\pm} \rightarrow e^{\pm} + \nu_{\mu} + \nu_{e}$$ Home Frame is the rest frame of the sterile neutrino, Other Frame is the rest frame of the pion, Other Other Frame is rest frame of the muon. We're interested in the muon neutrino and electron neutrino produced by the decay of the muon, which have the same spectra.

Initial part of Decay 4: $$\nu_s \rightarrow \pi^{\pm} + \mu^{\mp}$$ Home Frame is simply the rest frame of the sterile neutrino, no Other Frame is considered.

Decay 4a: $$\nu_s \rightarrow \pi^{\pm} + \mu^{\mp} \rightarrow e^{\pm} + \nu_{\mu} + \nu_{e}$$ Home Frame is the rest frame of the sterile neutrino, Other Frame is the rest frame of the muon. We're interested in the muon neutrino and electron neutrino produced by the decay of the muon, which have the same spectra.

Decay 4b: $$\nu_s \rightarrow \pi^{\pm} + \mu^{\mp} \rightarrow \nu_{\mu} + \mu^{\pm} $$ Home Frame is the rest frame of the sterile neutrino, and the Other Frame is the rest frame of the pion. We're interested in the muon neutrino and produced by the decay of the pion.

Decay 4c: $$\nu_s \rightarrow \pi^{\pm} + \mu^{\mp} \rightarrow \nu_{\mu} + \mu^{\pm} \rightarrow e^{\pm} + \nu_{\mu} + \nu_{e}$$ Home Frame is the rest frame of the sterile neutrino, Other Frame is the rest frame of the pion, Other Other Frame is rest frame of the muon. We're interested in the muon neutrino and electron neutrino produced by the decay of the muon, which have the same spectra.

* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
    * ``a_init``: The initial scale factor for this iteration of ``driver``
    * ``y_init``: The initial y array for this iteration of ``driver``
    * ``e_array``: Epsilon array...
    * ``eps_small``
    * ``eps_buffer``
    * ``dx``: Starting step size for ODE solver
    * ``N_steps``: Number of steps that will be saved by ODE solver
    * ``dN_steps``: Number of steps between saved steps for ODE solver
    * ``pl_last``: This defaults to False, but if it is set to True, then ``driver`` graphs two graphs: one is the comparison of the current neutrino occupation fraction to the initial neutrino occupation fraction from ``y_init``, and the other compares the derivatives (of $f$) to the changes in $f$ from $\nu-\nu$ collisions. These kind of sound like precursors to the graphs made in MakeMovie.py...
    * ``first``: Defaults to False. If it is set to True, ``k`` is just set to ``[0,0,0]``, otherwise ``find_breaks`` is used to calculate ``k``. I'm pretty sure ``first`` denotes if this is the first iteration of ``driver``. If it is (and thus ``k[0]`` $= 0$), ``make_der`` just calls ``coll.C`` directly rather than using the array of k values to try and shorten $\nu-\nu$ collision calculations in this first iteration of driver.
    * ``temp_fin``: Defaults to 0, but denotes a temperature at wish we want the code to break and just return the results. 
* **Calculations**:
    * ``A_model``: The first of two fitting parameters from ``ca.model_An`` used to approximate $nu-e$ collisions in $ \displaystyle \left. \frac{df}{da} \right\vert_{\rm \nu \text{-} e^{\pm} \, scattering} \approx -A n_e G_{\rm F}^2 p^n T^{2-n} \left(f-f_{\rm eq}\right) $
    * ``n_model``: The second of two fitting parameters from ``ca.model_An`` used to approximate $nu-e$ collisions in $ \displaystyle \left. \frac{df}{da} \right\vert_{\rm \nu \text{-} e^{\pm} \, scattering} \approx -A n_e G_{\rm F}^2 p^n T^{2-n} \left(f-f_{\rm eq}\right) $
    * ``num`` : The total length of ``y_init``, including the final three indices that account for $Q$, $T$, and $t$.
    * ``d1r``: The rate of decay process 1, $\Gamma_1$, at the given sterile neutrino mass and mixing angle.
    * ``d2r``: The rate of decay process 2, $\Gamma_2$, at the given sterile neutrino mass and mixing angle.
    * ``d3r``: The rate of decay process 3, $\Gamma_3$, at the given sterile neutrino mass and mixing angle.
    * ``d4r``: The rate of decay process 4, $\Gamma_4$, at the given sterile neutrino mass and mixing angle.
    * ``E_B1``: The energy of the active neutrino decay product in the Home Frame, $E_{\nu1} = \frac{m_s}{2}$ 
    * ``E_B2``: The energy of the active neutrino decay product in the Home Frame, $E_{\nu2} = \frac{m_s^2 - m_{\pi^0}}{2m_s}$
* **Calculations referring to initial part of decay 3**:
    * ``E_pi3``: Energy of the charged pion from decay 3 in the Home Frame, $E_{\pi^{\pm}_3} = \frac{m_s^2 + m_{\pi^{\pm}}^2 - m_e^2}{2m_s}$
    * ``p_pi3``: Momentum of the charged pion from decay 3 in the Home Frame, $p_{\pi^{\pm}_3} = \sqrt{E_{\pi^{\pm}_3}^2 - m_{\pi^{\pm}}^2}$
    * ``v3``: The speed of the pion rest frame relative to the sterile neutrino rest frame, $v_3 = p_{\pi^{\pm}_3}/E_{\pi^{\pm}_3}$
    * ``gammapi3``: The Lorentz factor of the pion's rest frame relative to the sterile neutrino rest frame, $\gamma_{\pi3} = E_{\pi^{\pm}_3}/m_{\pi^{\pm}} $
* **Calculations referring to decay 3a**:
    * ``E_B3``: Energy of the muon neutrino from the decay of the pion in the Other Frame (rest frame of the pion), $E_{B3} = \frac{m_{\pi^{\pm}}^2-m_{\mu}^2}{2m_{\pi^{\pm}}}$
    * ``E_B3max``: Maximum energy of the muon neutrino in the Home Frame, $E_{B3_{\rm max}} = \gamma_{\pi3} E_{B3}(1 + v_3)$
    * ``E_B3min``: Minimum energy of the muon neutrino in the Home Frame, $E_{B3_{\rm min}} = \gamma_{\pi3} E_{B3}(1 - v_3)$
* **Calculations referring to decay 3b**:    
    * ``E_mu3``: The energy of the muon from the decay of the pion in the Other Frame (rest frame of the pion), $E_{\mu_3} = m_{\pi^{\pm}} - E_{B3} = \frac{m_{\pi^{\pm}}^2+m_{\mu}^2}{2m_{\pi^{\pm}}}$
    * ``p_mu3``: The momentum of the muon from the decay of the pion in the Other Frame (rest frame of the pion), $p_{\mu_3} = \sqrt{E_{\mu_3}^2 - m_{\mu}^2}$
    * ``E_mumax3``: Maximum energy of the muon in the Home Frame, $E_{{\mu}_{3 \rm max}} = \gamma_{\pi3}(E_{\mu_3} + (v_3 p_{\mu_3}))$
    * ``E_mumin3``: Minimum energy of the muon in the Home Frame, $E_{{\mu}_{3 \rm min}} = \gamma_{\pi3}(E_{\mu_3} - (v_3 p_{\mu_3}))$
* **Calculations referring to initial part of decay 4**:
    * ``E_pi4``: Energy of the charged pion from decay 4 in the Home Frame, $E_{\pi^{\pm}_4} = \frac{m_s^2 + m_{\pi^{\pm}}^2 - m_{\mu}^2}{2m_s}$
    * ``p_pi4``: Momentum of the charged pion from decay 4 in the Home Frame, $p_{\pi^{\pm}_4} = \sqrt{E_{\pi^{\pm}_4}^2 - m_{\pi^{\pm}}^2}$
    * ``v4``: The speed of the pion rest frame relative to the sterile neutrino rest frame, $v_4 = p_{\pi^{\pm}_4}/E_{\pi^{\pm}_4}$
    * ``gammapi4``: The Lorentz factor of the pion's rest frame reltaive to the sterile neutrino rest frame, $\gamma_{\pi4} = E_{\pi^{\pm}_4}/m_{\pi^{\pm}} $
    * ``E_mu4i``: Energy of the muon produced directly by the decay of the sterile neutrino, $E_{\mu_{4i}} = m_s - E_{\pi^{\pm}_4}$
* **Calculations referring to decay 4a**:    
* **Calculations referring to decay 4b**:    
    * ``E_B4``: Energy of the muon neutrino from the decay of the pion in the Other Frame (rest frame of the pion), $E_{B4} = \frac{m_{\pi^{\pm}}^2-m_{\mu}^2}{2m_{\pi^{\pm}}}$
    * ``E_B4max``: Maximum energy of the muon neutrino in the Home Frame, $E_{B4_{\rm max}} = \gamma_{\pi4} E_{B4}(1 + v_4)$
    * ``E_B4min``: Minimum energy of the muon neutrino in the Home Frame, $E_{B4_{\rm min}} = \gamma_{\pi4} E_{B4}(1 - v_4)$
* **Calculations referring to decay 4c**:    
    * ``E_mu4ii``: The energy of the muon from the decay of the pion in the Other Frame (rest frame of the pion), $E_{\mu_{4ii}} = m_{\pi^{\pm}} - E_{B4} = \frac{m_{\pi^{\pm}}^2+m_{\mu}^2}{2m_{\pi^{\pm}}}$ 
    * ``p_mu4ii``: The momentum of the muon from the decay of the pion in the Other Frame (rest frame of the pion), $p_{\mu_{4ii}} = \sqrt{E_{\mu_{4ii}}^2 - m_{\mu}^2}$
    * ``E_mumax4``: Maximum energy of the muon in the Home Frame, $E_{{\mu}_{4 \rm max}} = \gamma_{\pi4}(E_{\mu_{4ii}} + (v_4 p_{\mu_{4ii}}))$
    * ``E_mumin4``: Minimum energy of the muon in the Home Frame, $E_{{\mu}_{4 \rm min}} = \gamma_{\pi4}(E_{\mu_{4ii}} - (v_4 p_{\mu_{4ii}}))$
* **Functions**
* ### ``C_ve`` 
A function that approximates the change in $f$ due to neutrino-electron collisions. At the beginning of each iteration of ``driver``, the parameters ``A_model`` and ``n_model`` are recalculated to best fit the current collision integral, but doing this (and thus calculating the full collision integral from ``nu_e_collisions.py``) only once per driver iteration saves significant time. 
* **Inputs**: 
    * ``p_array``: Current neutrino momenta array, equivalent to ``e_array``/``Tcm``.
    * ``Tcm``: Current comomving temperature, $T_{\rm cm} = 1/a$
    * ``T``: Current plasma temperature
    * ``f``: Current array of neutrino occupation fractions
* **Outputs**: 
    * The approximation for the electron-neutrino collisions whose two fitting parameters are $A$ (``A_model``) and $n$ (``n_model``), which are calculated earlier in the function ``driver``: $ \displaystyle \left. \frac{df}{da} \right\vert_{\rm \nu \text{-} e^{\pm} \, scattering} \approx -A n_e G_{\rm F}^2 p^n T^{2-n} \left(f-f_{\rm eq}\right) $
* ### ``make_der``
This function makes the function that is sent to ``DES.destination_x_dx``, and it exists solely to alter the way the $\nu-\nu$ collision integrals are calculated in the ``derivatives function`` and return ``derivatives`` accordingly.
* **Inputs**
    * ``kk``: An array of length three that comes from the ``find_breaks`` function, or is simply ``[0,0,0]`` if this is the first iteration of ``driver``. Generally, ``k[0]`` is the index after the final ``f`` index where ``f``<$1 \times 10^{-20}$, ``k[1]`` is the index after the final ``f`` index where ``f``<$1 \times 10^{-25}$, and ``k[2]`` is the index after the final ``f`` index where ``f``<$1 \times 10^{-30}$. There are exceptions that make these k values bigger if they are too close to the indices that correspond to the energies of neutrinos from decays 1 and 2, see ``find_breaks`` info above. 
* **Outputs**
    * ``derivatives``: The derivatives function that will be used by ``DES.destination_x_dx``, see more info below.
* ### ``derivatives``
* **Inputs**: 
    * ``a``: Scale factor from previous time step 
    * ``y``: All dependent variables ($N_E$, temperature, and time) from previous time step
* **Calculations**
    * ``dtda``: Need to change this now that $\epsilon$ is introduced $$ \frac{dt}{da} = \frac{m_{\rm Pl}}{2 a} \frac{1}{\sqrt{\cfrac{2\pi}{3} \left( \cfrac{T^4 \pi^2}{15} + \cfrac{2T^4 \text{``I1``}}{\pi^2} + \cfrac{7 \pi^2 T_{\rm cm}^4}{40} + m_sn_s +  {\displaystyle \int } \cfrac{df}{da} \cfrac{1}{E^2 a^3} \right)}} $$
    * ``d3b_e0``: The contribution to $df/da$ at $\epsilon=0$ from decay process 3b. We have to do this separately because otherwise we would divide by 0... $ \displaystyle \left. \frac{df}{da}_{3b} \right\vert_{\epsilon=0} 2*(1-\epsilon_e^2) \Gamma_3 \gamma_{\pi3} m_{\mu}^2 G_F^2 E_{\mu_3} n_s \frac{dt}{da}\frac{1}{\pi \Gamma_{\mu_b}'} $
    * ``d4b_e0``: The contribution to $df/da$ at $\epsilon=0$ from decay process 4b (4a??). We have to do this separately because otherwise we would divide by 0... $ \displaystyle \left. \frac{df}{da}_{4b} \right\vert_{\epsilon=0} 2*(1-\epsilon_e^2) \Gamma_4 \frac{E_{\mu}}{m_{\mu}} m_{\mu}^2 G_F^2 n_s \frac{dt}{da}\frac{1}{\pi \Gamma_{\mu_b}'} $
    * ``d4c_e0``: The contribution to $df/da$ at $\epsilon=0$ from decay process 4c. We have to do this separately because otherwise we would divide by 0... $ \displaystyle \left. \frac{df}{da}_{4c} \right\vert_{\epsilon=0} 2*(1-\epsilon_e^2) \Gamma_4 \gamma_{\pi4} m_{\mu}^2 G_F^2 E_{\mu_4} n_s \frac{dt}{da}\frac{1}{\pi \Gamma_{\mu_b}'} $
    * ``d_array[0]``: We have to calculate ``d_array[0]`` outside of the main for loop because we divide by $\epsilon$ in our typical calculations (in Gam_mua I think?). The only contributors to $df/da$ at $\epsilon=0$ are decay processes 3b, 4b, and 4c, because the others don't have neutrinos at energies of 0 as possible outputs due to conservation of energy. So, $\left. \frac{df}{da} \right\vert_{\epsilon=0} = \left. \frac{df}{da}_{3b} \right\vert_{\epsilon=0} + \left. \frac{df}{da}_{4b} \right\vert_{\epsilon=0} + \left. \frac{df}{da}_{4c} \right\vert_{\epsilon=0} $.
    * ``c``: This is simply the contribution to $df/da$ from both the neutrino-neutrino and neutrino-electron collisions. If ``kk[0]`` = 0, then this is the first iteration of ``driver`` and we don't want to over-approximate our $\nu-\nu$ collision integral, so we just call ``C`` directly from ``nu_nu_collisions.py``, which returns the collisions integral $I$. Otherwise, we call ``C_short`` is called, which also calculates $I$ but sort of takes shortcuts to do so (read the documentation on this function for more info). Then, to add the contribution to $df/da$ from $\nu-e$ collisions, we call the function ``C_ve``, which returns an approximation for $df/da$ that is recreated every time driver is called. In total, $ \displaystyle c = \left(\frac{df}{da}_{\nu-\nu} + \frac{df}{da}_{\nu-e} \right) \frac{dt}{da} $.
    * ``d1``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 1 in time interval $dt$, $$\left(\frac{dP}{dt \, dE}\right)_1 = \Gamma_1 \delta(E-E_{\nu1}) $$
    * ``d2``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 2 in time interval $dt$, $$\left(\frac{dP}{dt \, dE}\right)_2 = \Gamma_2 \delta(E-E_{\nu2}) .$$ 
    * ``d3a``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 3a in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process the other half of the time, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{3a} = \frac{\Gamma_3}{2}  \dfrac{1}{2\gamma_{\pi3} v_3 E_{B3}} \cases{1 \text{ if } E_{B3_{\rm min}}<E<E_{B3_{\rm max}} \\ 0 \text{ otherwise}} $$
    * ``d3b``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 3b in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{3b} = \frac{\Gamma_3}{2 \gamma_{\pi3} v_3 p_{\mu3} \Gamma_{\mu_b}'} \int_{E_{\mu \rm min}}^{E_{\mu \rm max}} \frac{\Gamma_{\mu_a}'}{2\gamma_{\mu}v_{\mu}} dE_{\mu} $$
    * ``Gam_mua``: Decay rate of muon in the rest frame of the muon and the spectra of neutrinos created by this decay in the rest frame of the muon, $\Gamma_{\mu_a}' = \int_a^b \frac{d\Gamma_{\mu}'}{dE_{\nu}'}\frac{dE_{\nu}'}{E_{\nu}'}$ where $a = E/(\gamma_{\pi4}(1+v_4))$ and $b= {\rm min}(E_{\nu \rm max},E/(\gamma_{\pi4}(1-v_4)))$
    * ``d4a``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 4a in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{4a} = \frac{\Gamma_4 \Gamma_{\mu_a}'}{2\gamma_{\pi4} v_4 \Gamma_{\mu_b}'}$$
    * ``d4b``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 4b in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process the other half of the time, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{4b} = \frac{\Gamma_4}{2}\dfrac{1}{2\gamma_{\pi4} v_4 E_{B4}} \cases{1 \text{ if } E_{B4_{\rm min}}<E<E_{B4_{\rm max}} \\ 0 \text{ otherwise}} $$ 
    * ``d4c``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 4c in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{4c} = \frac{\Gamma_4}{2 \gamma_{\pi4} v_4 p_{\mu4} \Gamma_{\mu_b}'} \int_{E_{{\mu}_{4 \rm min}}}^{E_{{\mu}_{4 \rm max}}} \frac{\Gamma_{\mu_a}'}{2\gamma_{\mu}v_{\mu}} dE_{\mu} $$
    * ``d_array`` : I'm not sure if this is df/da or f or maybe dN/da?? $ \displaystyle \frac{df}{da} = \frac{2\pi^2}{\epsilon^2 T_{\rm cm}^2 a^3} \big( \left(\frac{dP}{dt \, dE}\right)_{1} + \left(\frac{dP}{dt \, dE}\right)_{2} + \left(\frac{dP}{dt \, dE}\right)_{3a} + \left(\frac{dP}{dt \, dE}\right)_{3b} + \left(\frac{dP}{dt \, dE}\right)_{4a} + \left(\frac{dP}{dt \, dE}\right)_{4b} + \left(\frac{dP}{dt \, dE}\right)_{4c} \big) n_s a^3 \frac{dt}{da} + \big(C_{\nu-\nu} + C_{\nu-e} \big) \frac{dt}{da} $
    * ``df_array`` : Integrated over in ``dQda``: $\displaystyle \frac{df}{da} \frac{\epsilon^3}{2\pi^2}$
    * ``dQda`` : The change in heat (?) with respect to scale factor, used to calculate $dT/da$, $\displaystyle \frac{dQ}{da} = \frac{m_s n_s a^3 \frac{dt}{da}}{\tau_s} - T_{\rm cm}^4 a^3 \int \frac{df}{da} \frac{\epsilon^3}{2\pi^2}d\epsilon $
    * ``dTda`` :  Need to change (apparently first term is now $\frac{1}{T} \frac{dQ}{da}$, $$\frac{dT}{da} = \cfrac{ \cfrac{f_T m_s a^3}{\tau_s T} n_s \cfrac{dt}{da} -3a^2T^3 \left[ \cfrac{4\pi^2}{45} + \cfrac{2}{\pi^2} \left( \text{``I1``} + \cfrac{1}{3} \text{``I2``} \right) \right]}{3T^2a^3 \left[ \cfrac{4\pi^2}{45} + \cfrac{2}{\pi^2} \left( \text{``I1``} + \cfrac{1}{3} \text{``I2``} \right) \right] - \cfrac{2 m_e^2 a^3}{\pi^2} \left( \text{``dI1``} + \cfrac{1}{3} \text{``dI2``} \right)} $$ 
* **Outputs**: 
    * ``d_array``: Derivatives at this previous time step that will be used to calculate ``a`` and ``y`` for the next time step  
    
* **Calculations**
    * ``a0``: Initially set to ``a_init``, but with each iteration of ``DES.destination_x_dx``, gets set to the most recent value of $a$, ``a_array[-1]``.
    * ``y0``: Initially set to ``y_init``, but with each iteration of ``DES.destination_x_dx``, gets set to the most recent value of $a$, ``y_matrix[:,-1]``.
    * ``f_FULL``: Not sure what this is exactly to be honest, but it is used to determine if ``decay_on`` should be set to ``False``. Ultimately, I think the idea is that as $n_s$ gets too small, then ``decay_on`` gets set to ``False`` because there are so few sterile neutrinos left, but I don't know why the rest of this stuff is here... ``f_FULL`` $= \displaystyle \frac{2 \pi^2 a_0 n_s}{E_{B_2}^2 (\text{e_array[-1]}-\text{e_array[-2]})}$
    * ``decay_on``: A boolean set to ``True`` initially, but turned off if ``f_FULL``<``f_BUFFER``. Used to set to ``a_max`` to a different value if ``True``, and returned by ``driver`` and used in ``forward`` to...
    * ``a_max``: Initially set to $a_{\rm max} = a_0 * a_{\rm MAXMULT}$, but changed to ``min(e_array[-MIN_eps_BUFFER-1] / E_B1, a_max)`` if ``decay_on`` is ``True``. I think the verbiage of "max" is a bit confusing, as far as I can tell, this is the maximum value of ``a`` *for this iteration of driver*, such that when ``a``>``a_max``, we break out of ``DES.destination_x_dx`` and ``driver``, sent back to one of the outside functions (not sure which one yet), which then calls driver *again*, thus setting a new model for $\nu-e$ collisions, a new ``a_max``, etc. Thus, ``a_max`` is designed to be big enough that we don't have to recalculate $\nu-e$ collisions a billion times, but small enough that we don't miss important data where we should be recalculating $\nu-e$ collisions. Basically, this is not the maximum value for ``a`` overall (which I think we've generally been using $a_{\rm max}=2$), but rather the max value of $a$ for this one iteration of ``driver``. 
    * ``a_results``: Contains all the ``a`` results as calculated by ``DES.destination_x_dx``; ``a_array[-1]`` is appended to the end of ``a_results`` after each run of ``DES.destination_x_dx`` (uh, why not all of ``a_array``?).
    * ``y_results``: Contains all the ``y`` results as calculated by ``DES.destination_x_dx``; ``y_matrix[:,-1]`` is appended to the end of ``y_results`` after each run of ``DES.destination_x_dx`` (uh, why not all of ``y_matrix``?).
    * ``k``: An array of length three that comes from the ``find_breaks`` function, or is simply ``[0,0,0]`` if this is the first iteration of ``driver``. Generally, ``k[0]`` is the index after the final ``f`` index where ``f``<$1 \times 10^{-20}$, ``k[1]`` is the index after the final ``f`` index where ``f``<$1 \times 10^{-25}$, and ``k[2]`` is the index after the final ``f`` index where ``f``<$1 \times 10^{-30}$. There are exceptions that make these k values bigger if they are too close to the indices that correspond to the energies of neutrinos from decays 1 and 2, see ``find_breaks`` info above. 
    * ``a_array``: The output of all saved $a$ values from ``DES.destination_x_dx``. It seems like we only save the final value of ``a_array`` to ``a_results`` which kind of strikes me as interesting, but I guess we don't really need a ton of points to get the picture of what's happeneing so long as those points are accurate! 
    * ``y_matrix``: The output of all saved $f$, $Q$, $T$, and $t$ values from ``DES.destination_x_dx``. It seems like we only save the final array of ``y_matrix`` to ``y_results`` which kind of strikes me as interesting, but I guess we don't really need a ton of points to get the picture of what's happeneing so long as those points are accurate! 
    * ``dx_new``: The most recent stepsize used by ``DES.destination_x_dx``.
    * ``dx``: Stepsize reset to ``dx_new`` if we accept the most recent run of ``DES.destination_x_dx``.
    * ``eps``: The index of the final place that ``y0`` > ``f_SMALL`` (does the code assume it's an actual epsilon value? hmm...). Used in an if statement to determine if the for loop should be broken out of...
    
### ``time_derivative``
This function simply returns $dt/da$, but will only work for the standard cosmological model as sterile neutrinos and sterile neutrino decay products are not included.
* **Imports**
    * ``a``: Current scale factor
    * ``T``: Current plasma temperature
* **Outputs**
    * ``dtda``: The derivative of time with respect to scale factor, $ \displaystyle \frac{dt}{da} = \frac{\dfrac{m_{PL}}{2a}}{\sqrt{\dfrac{2\pi}{3}* \bigg(\dfrac{T^4 \pi^2}{15} + \dfrac{2T^4 ``I1``}{\pi^2} + \dfrac{7 \pi^2}{40 a^4} \bigg)}} $